### Exploring how team attributes impact scores in the premier league
# approach is merge the match and (winning) team data to evaluate which team features most impact 

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sqlite3 as sql
import matplotlib.pyplot as plt

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

/Users/kofi/anaconda/envs/python3/lib/python3.4/site-packages/matplotlib/font_manager.py:280: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  'Matplotlib is building the font cache using fc-list. '


database.sqlite



In [2]:
# create the connection to the database
con = None
con = sql.connect('../input/database.sqlite')
# create the cursor
cur= con.cursor()
# select only the information for the EPL
#   - changed from sqlite interface to use pd.read_sql
query = "select * from League where name like '%England%'"
#eplinfo = cur.execute("select * from League where name like '%England%'").fetchall()
eplinfo = pd.read_sql(query,con=con) 
#print(type(eplinfo))
print(eplinfo)
query = "select * from League"
leagues = pd.read_sql(query,con=con) #cur.execute().fetchall()
#print(type(leagues))
print(leagues)


     id  country_id                    name
0  1729        1729  England Premier League
       id  country_id                      name
0       1           1    Belgium Jupiler League
1    1729        1729    England Premier League
2    4769        4769            France Ligue 1
3    7809        7809     Germany 1. Bundesliga
4   10257       10257             Italy Serie A
5   13274       13274    Netherlands Eredivisie
6   15722       15722        Poland Ekstraklasa
7   17642       17642  Portugal Liga ZON Sagres
8   19694       19694   Scotland Premier League
9   21518       21518           Spain LIGA BBVA
10  24558       24558  Switzerland Super League


### Get match information 
filter on epl and get the actual team names
also select date, season, date and time, home team goal, away team goals, home team wins?

In [3]:
season = "2010/2011"
# Set up the query to join the match information with the home and away team IDS
#query = "Select match_api_id, m.home_team_api_id, m.away_team_api_id, \
#        m.stage, m.date, t1.team_long_name as home_team, \
#        t2.team_long_name as away_team, m.home_team_goal, m.away_team_goal \
#        from Match as m join Team as t1 join Team as t2 \
#        on t1.team_api_id = m.home_team_api_id and t2.team_api_id = m.away_team_api_id  \
#        where m.league_id = {} and m.season='{}'".format(eplinfo.id[0],season)
#print(query)
# get matches
#m.id, m.match_api_id, m.home_team_api_id, m.away_team_api_id, \
#        m.stage, m.date, m.home_team_goal, m.away_team_goal from Match as'
query = "Select * from Match \
        where league_id = {} and season='{}'".format(eplinfo.id[0],season)
matches = pd.read_sql(query,con=con)
matches = matches[matches.columns[:11]]
matches.head()
# get teams
query = "SELECT * FROM Team"
teams = pd.read_sql(query,con=con)

query = "SELECT * FROM Team_Attributes where date >= '2010-07-01 00:00:00' and date <='2011-06-0100:00:00'"
team_attributes = pd.read_sql(query,con=con)
#print(team_attributes.shape)
#team_attributes.head()




In [4]:
# pulled from Pete Hodges's kernel 'https://www.kaggle.com/petehodge/d/hugomathien/soccer/epl-weekly-predicting
matches = pd.merge(left=matches, right=teams, how='left', left_on='home_team_api_id', right_on='team_api_id')
matches = matches.drop(['country_id','league_id','id_y','team_api_id','team_fifa_api_id','team_short_name'], axis=1)
matches.rename(columns={'id_x':'match_id','date':'match_date','team_long_name':'home_team'}, inplace=True)
matches = pd.merge(left=matches, right=teams, how='left', left_on='away_team_api_id', right_on='team_api_id')
matches = matches.drop(['id', 'match_api_id', 'team_fifa_api_id', 'team_short_name'], axis=1)

matches.rename(columns={'team_long_name':'away_team'}, inplace=True)
matches.head()



,match_id,season,stage,match_date,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,home_team,team_api_id,away_team
0,2489,2010/2011,1,2010-08-14 00:00:00,10252,8654,3,0,Aston Villa,8654,West Ham United
1,2490,2010/2011,1,2010-08-14 00:00:00,8655,8668,1,0,Blackburn Rovers,8668,Everton
2,2491,2010/2011,1,2010-08-14 00:00:00,8559,9879,0,0,Bolton Wanderers,9879,Fulham
3,2492,2010/2011,1,2010-08-14 00:00:00,8455,8659,6,0,Chelsea,8659,West Bromwich Albion
4,2493,2010/2011,1,2010-08-15 00:00:00,8650,9825,1,1,Liverpool,9825,Arsenal


In [5]:
# do the same for the team attributes for the home and away teams
matches = pd.merge(left=matches, right=team_attributes, how='left', left_on='home_team_api_id', right_on='team_api_id')
matches = matches.drop(['id', 'team_fifa_api_id', 'team_api_id_x','team_api_id_y','date'], axis=1)
matches.rename(columns={'buildUpPlaySpeed':'home_buildUpPlaySpeed','buildUpPlaySpeedClass':'home_buildUpPlaySpeedClass',
                        'buildUpPlayDribbling':'home_buildUpPlayDribbling',
                        'buildUpPlayDribblingClass':'home_buildUpPlayDribblingClass',
                        'buildUpPlayPassing':'home_buildUpPlayPassing','buildUpPlayPassingClass':'home_buildUpPlayPassingClass',
                        'buildUpPlayPositioningClass':'home_buildUpPlayPositioningClass',
                        'chanceCreationPassing':'home_chanceCreationPassing','chanceCreationPassingClass':'home_chanceCreationPassingClass',
                        'chanceCreationCrossing':'home_chanceCreationCrossing',
                        'chanceCreationCrossingClass':'home_chanceCreationCrossingClass','chanceCreationShooting':'home_chanceCreationShooting',
                        'chanceCreationShootingClass':'home_chanceCreationShootingClass','chanceCreationPositioningClass':'home_chanceCreationPositioningClass','defencePressure':'home_defencePressure',
                        'defencePressureClass':'home_defencePressureClass','defenceAggression':'home_defenceAggression',
                        'defenceAggressionClass':'home_defenceAggressionClass','defenceTeamWidth':'home_defenceTeamWidth',
                        'defenceTeamWidthClass':'home_defenceTeamWidthClass','defenceDefenderLineClass':'home_defenceDefenderLineClass'}, inplace=True)

matches = pd.merge(left=matches, right=team_attributes, how='left', left_on='away_team_api_id', right_on='team_api_id')
matches = matches.drop(['id', 'team_fifa_api_id', 'team_api_id', 'date'], axis=1)

matches.rename(columns={'buildUpPlaySpeed':'away_buildUpPlaySpeed','buildUpPlaySpeedClass':'away_buildUpPlaySpeedClass',
                        'buildUpPlayDribbling':'away_buildUpPlayDribbling',
                        'buildUpPlayDribblingClass':'away_buildUpPlayDribblingClass',
                        'buildUpPlayPassing':'away_buildUpPlayPassing','buildUpPlayPassingClass':'away_buildUpPlayPassingClass',
                        'buildUpPlayPositioningClass':'away_buildUpPlayPositioningClass',
                        'chanceCreationPassing':'away_chanceCreationPassing','chanceCreationPassingClass':'away_chanceCreationPassingClass',
                        'chanceCreationCrossing':'away_chanceCreationCrossing',
                        'chanceCreationCrossingClass':'away_chanceCreationCrossingClass','chanceCreationShooting':'away_chanceCreationShooting',
                        'chanceCreationShootingClass':'away_chanceCreationShootingClass','chanceCreationPositioningClass':'away_chanceCreationPositioningClass','defencePressure':'away_defencePressure',
                        'defencePressureClass':'away_defencePressureClass','defenceAggression':'away_defenceAggression',
                        'defenceAggressionClass':'away_defenceAggressionClass','defenceTeamWidth':'away_defenceTeamWidth',
                        'defenceTeamWidthClass':'away_defenceTeamWidthClass','defenceDefenderLineClass':'away_defenceDefenderLineClass'}, inplace=True)
print(matches.columns)


matches["home_team_points"] = 3*(matches["home_team_goal"] > matches["away_team_goal"]) + 1*(matches["home_team_goal"] == matches["away_team_goal"])
#matches["home_team_points"] = 1*(matches["home_team_goal"] == matches["away_team_goal"])
#print(matches.shape)
matches.head(10)

Index(['match_id', 'season', 'stage', 'match_date', 'home_team_api_id',
       'away_team_api_id', 'home_team_goal', 'away_team_goal', 'home_team',
       'away_team', 'home_buildUpPlaySpeed', 'home_buildUpPlaySpeedClass',
       'home_buildUpPlayDribbling', 'home_buildUpPlayDribblingClass',
       'home_buildUpPlayPassing', 'home_buildUpPlayPassingClass',
       'home_buildUpPlayPositioningClass', 'home_chanceCreationPassing',
       'home_chanceCreationPassingClass', 'home_chanceCreationCrossing',
       'home_chanceCreationCrossingClass', 'home_chanceCreationShooting',
       'home_chanceCreationShootingClass',
       'home_chanceCreationPositioningClass', 'home_defencePressure',
       'home_defencePressureClass', 'home_defenceAggression',
       'home_defenceAggressionClass', 'home_defenceTeamWidth',
       'home_defenceTeamWidthClass', 'home_defenceDefenderLineClass',
       'away_buildUpPlaySpeed', 'away_buildUpPlaySpeedClass',
       'away_buildUpPlayDribbling', 'away_buildUpPl

,match_id,season,stage,match_date,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,home_team,away_team,...,away_chanceCreationShootingClass,away_chanceCreationPositioningClass,away_defencePressure,away_defencePressureClass,away_defenceAggression,away_defenceAggressionClass,away_defenceTeamWidth,away_defenceTeamWidthClass,away_defenceDefenderLineClass,home_team_points
0,2489,2010/2011,1,2010-08-14 00:00:00,10252,8654,3,0,Aston Villa,West Ham United,...,Normal,Organised,45,Medium,45,Press,50,Normal,Cover,3
1,2490,2010/2011,1,2010-08-14 00:00:00,8655,8668,1,0,Blackburn Rovers,Everton,...,Normal,Organised,55,Medium,65,Press,60,Normal,Cover,3
2,2491,2010/2011,1,2010-08-14 00:00:00,8559,9879,0,0,Bolton Wanderers,Fulham,...,Normal,Organised,45,Medium,35,Press,50,Normal,Cover,1
3,2492,2010/2011,1,2010-08-14 00:00:00,8455,8659,6,0,Chelsea,West Bromwich Albion,...,Normal,Organised,35,Medium,55,Press,60,Normal,Cover,3
4,2493,2010/2011,1,2010-08-15 00:00:00,8650,9825,1,1,Liverpool,Arsenal,...,Normal,Free Form,50,Medium,40,Press,45,Normal,Cover,1
5,2494,2010/2011,1,2010-08-16 00:00:00,10260,10261,3,0,Manchester United,Newcastle United,...,Normal,Organised,45,Medium,45,Press,50,Normal,Cover,3
6,2495,2010/2011,1,2010-08-14 00:00:00,8472,8658,2,2,Sunderland,Birmingham City,...,Normal,Organised,30,Deep,60,Press,45,Normal,Cover,1
7,2496,2010/2011,1,2010-08-14 00:00:00,8586,8456,0,0,Tottenham Hotspur,Manchester City,...,Lots,Free Form,65,Medium,65,Press,50,Normal,Cover,1
8,2497,2010/2011,1,2010-08-14 00:00:00,8602,10194,2,1,Wolverhampton Wanderers,Stoke City,...,Normal,Organised,25,Deep,65,Press,50,Normal,Cover,3
9,2498,2010/2011,1,2010-08-14 00:00:00,8528,8483,0,4,Wigan Athletic,Blackpool,...,Normal,Organised,45,Medium,50,Press,45,Normal,Cover,0


In [78]:
print(matches.shape)

(380, 53)


In [79]:
matches.describe()

,match_id,stage,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,home_buildUpPlaySpeed,home_buildUpPlayPassing,home_chanceCreationPassing,home_chanceCreationCrossing,...,home_defenceTeamWidth,away_buildUpPlaySpeed,away_buildUpPlayPassing,away_chanceCreationPassing,away_chanceCreationCrossing,away_chanceCreationShooting,away_defencePressure,away_defenceAggression,away_defenceTeamWidth,home_team_points
count,380.000000,380.000000,380.00000,380.00000,380.000000,380.000000,380.000000,380.000000,380.000000,380.000000,...,380.000000,380.000000,380.000000,380.000000,380.000000,380.000000,380.000000,380.000000,380.000000,380.000000
mean,2678.500000,19.500000,9037.80000,9037.80000,1.623684,1.173684,56.250000,53.250000,50.750000,56.750000,...,54.000000,56.250000,53.250000,50.750000,56.750000,54.500000,43.000000,51.750000,54.000000,1.705263
std,109.840794,10.980313,714.60605,714.60605,1.240464,1.114236,10.486988,11.555904,12.291668,10.769993,...,7.009229,10.486988,11.555904,12.291668,10.769993,12.949567,9.285845,8.711326,7.009229,1.275908
min,2489.000000,1.000000,8455.00000,8455.00000,0.000000,0.000000,35.000000,30.000000,30.000000,30.000000,...,45.000000,35.000000,30.000000,30.000000,30.000000,35.000000,25.000000,35.000000,45.000000,0.000000
25%,2583.750000,10.000000,8551.25000,8551.25000,1.000000,0.000000,50.000000,50.000000,40.000000,50.000000,...,50.000000,50.000000,50.000000,40.000000,50.000000,45.000000,35.000000,45.000000,50.000000,1.000000
50%,2678.500000,19.500000,8654.50000,8654.50000,1.000000,1.000000,57.500000,52.500000,52.500000,57.500000,...,50.000000,57.500000,52.500000,52.500000,57.500000,52.500000,45.000000,55.000000,50.000000,1.000000
75%,2773.250000,29.000000,9838.50000,9838.50000,2.000000,2.000000,61.250000,60.000000,60.000000,65.000000,...,60.000000,61.250000,60.000000,60.000000,65.000000,65.000000,45.000000,56.250000,60.000000,3.000000
max,2868.000000,38.000000,10261.00000,10261.00000,7.000000,6.000000,75.000000,80.000000,70.000000,70.000000,...,65.000000,75.000000,80.000000,70.000000,70.000000,80.000000,65.000000,65.000000,65.000000,3.000000


Set the match_id to be the dataset key.

It is redundant to keep the home and away teams goals in the analysis since the output (whether the home team wins, loses or draws) is directly dependent on both features. So we'll remove them from the data set or not add them into our feature set. Other fields to exclude from the feature set include the match date (for now), the names of the teams playing etc.

home_buildUpPlayDribbling and away_buildUpPlayDribbling have None entries


In [91]:
matches.index = matches['match_id']
# then drop the match_id and also drop stage for now
to_drop = ['match_id', 'stage', 'home_team_goal', 'away_team_goal', 'match_date','home_team_api_id',
           'away_team_api_id','home_team', 'away_team','season',
           'home_buildUpPlayDribbling','away_buildUpPlayDribbling']
# make a copy of the matches dataframe and drop the appropriate fields while deleting the unneeded features
matches_ml = matches.drop(to_drop, axis =1)
print(matches_ml.shape)


(380, 41)


Clean up the data by converting categorical data to their one-hot encoded forms

In [92]:
# Make a list of all columns with categorical data
cat_list= matches_ml.select_dtypes(include=['object']).columns.tolist()
#print(cat_list)

In [93]:
##print(matches_ml.columns)
#from sklearn.preprocessing import OneHotEncoder 
#from sklearn.preprocessing import LabelEncoder

## pulled in from http://stackoverflow.com/questions/24458645/label-encoding-across-multiple-columns-in-scikit-learn
#class MultiColumnLabelEncoder:
#    def __init__(self,columns = None):
#        self.columns = columns # array of column names to encode

#    def fit(self,X,y=None):
#        return self # not relevant here

#    def transform(self,X):
#        '''
#        Transforms columns of X specified in self.columns using
#        LabelEncoder(). If no columns specified, transforms all
#        columns in X.
#       '''
#        output = X.copy()
#        if self.columns is not None:
#            for col in self.columns:
#                #print(col)
#                output[col] = LabelEncoder().fit_transform(output[col])
#        else:
#            for colname,col in output.iteritems():
#                output[colname] = LabelEncoder().fit_transform(col)
#        return output

#    def fit_transform(self,X,y=None):
#        return self.fit(X,y).transform(X)


In [94]:
#MultiColumnLabelEncoder(columns = cat_list ).fit_transform(matches_ml)
#- Then convert those fields to encoded numeric forms
#http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html
matches_ml = pd.get_dummies(matches_ml, prefix=cat_list)
#print(new_matches.columns)

In [95]:
matches_ml.head()
#for i in [cat_list]:
#    print(matches_ml.iloc[[1]][i])
#    #print("field: {}, value: {}".format(i,v))
    

,home_buildUpPlaySpeed,home_buildUpPlayPassing,home_chanceCreationPassing,home_chanceCreationCrossing,home_chanceCreationShooting,home_defencePressure,home_defenceAggression,home_defenceTeamWidth,away_buildUpPlaySpeed,away_buildUpPlayPassing,...,away_chanceCreationCrossingClass_Normal,away_chanceCreationShootingClass_Lots,away_chanceCreationShootingClass_Normal,away_chanceCreationPositioningClass_Free Form,away_chanceCreationPositioningClass_Organised,away_defencePressureClass_Deep,away_defencePressureClass_Medium,away_defenceAggressionClass_Press,away_defenceTeamWidthClass_Normal,away_defenceDefenderLineClass_Cover
match_id,,,,,,,,,,,,,,,,,,,,,
2489,65,50,45,55,50,35,45,65,35,35,...,1,0,1,0,1,0,1,1,1,1
2490,55,65,50,50,35,45,60,55,60,50,...,0,0,1,0,1,0,1,1,1,1
2491,45,70,55,70,50,40,55,50,50,55,...,1,0,1,0,1,0,1,1,1,1
2492,60,50,60,60,80,35,55,50,60,65,...,1,0,1,0,1,0,1,1,1,1
2493,60,50,40,45,60,55,55,50,75,40,...,1,0,1,1,0,0,1,1,1,1


In [103]:
# move the home team points to the target field
y = matches_ml['home_team_points']
matches_ml.drop(['home_team_points'], axis=1)

,home_buildUpPlaySpeed,home_buildUpPlayPassing,home_chanceCreationPassing,home_chanceCreationCrossing,home_chanceCreationShooting,home_defencePressure,home_defenceAggression,home_defenceTeamWidth,away_buildUpPlaySpeed,away_buildUpPlayPassing,...,away_chanceCreationCrossingClass_Normal,away_chanceCreationShootingClass_Lots,away_chanceCreationShootingClass_Normal,away_chanceCreationPositioningClass_Free Form,away_chanceCreationPositioningClass_Organised,away_defencePressureClass_Deep,away_defencePressureClass_Medium,away_defenceAggressionClass_Press,away_defenceTeamWidthClass_Normal,away_defenceDefenderLineClass_Cover
match_id,,,,,,,,,,,,,,,,,,,,,
2489,65,50,45,55,50,35,45,65,35,35,...,1,0,1,0,1,0,1,1,1,1
2490,55,65,50,50,35,45,60,55,60,50,...,0,0,1,0,1,0,1,1,1,1
2491,45,70,55,70,50,40,55,50,50,55,...,1,0,1,0,1,0,1,1,1,1
2492,60,50,60,60,80,35,55,50,60,65,...,1,0,1,0,1,0,1,1,1,1
2493,60,50,40,45,60,55,55,50,75,40,...,1,0,1,1,0,0,1,1,1,1
2494,65,40,65,65,70,45,45,65,55,50,...,1,0,1,0,1,0,1,1,1,1
2495,55,55,55,65,55,45,55,65,45,60,...,1,0,1,0,1,1,0,1,1,1
2496,60,55,70,70,70,30,40,60,70,60,...,1,1,0,1,0,0,1,1,1,1
2497,55,55,60,60,45,45,45,50,50,80,...,0,0,1,0,1,1,0,1,1,1


In [104]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(matches_ml)

In [105]:
print("Feature space holds %d observations and %d features" % X.shape)
print("Unique target labels:", np.unique(y))

Feature space holds 380 observations and 63 features
Unique target labels: [0 1 3]


In [120]:
for c in matches_ml.columns:
    matches_ml[c].isnull().values.any()

In [121]:
from sklearn.cross_validation import KFold

def run_cv(X,y,clf_class,**kwargs):
    # Construct a kfolds object
    kf = KFold(len(y),n_folds=3,shuffle=True)
    y_pred = y.copy()
    
    # Iterate through folds
    for train_index, test_index in kf:
        X_train, X_test = X[train_index], X[test_index]
        y_train = y[train_index]
        # Initialize a classifier with key word arguments
        clf = clf_class(**kwargs)
        clf.fit(X_train,y_train)
        y_pred[test_index] = clf.predict(X_test)
    return y_pred

In [122]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier as RF
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.linear_model import LogisticRegression as LR
from sklearn.ensemble import GradientBoostingClassifier as GBC
from sklearn.metrics import average_precision_score

def accuracy(y_true,y_pred):
    # NumPy interpretes True and False as 1. and 0.
    return np.mean(y_true == y_pred)

print("Logistic Regression:")
print("%.3f" % accuracy(y, run_cv(X,y,LR)))
print("Gradient Boosting Classifier")
print("%.3f" % accuracy(y, run_cv(X,y,GBC)))
print("Support vector machines:")
print("%.3f" % accuracy(y, run_cv(X,y,SVC)))
print("Random forest:")
print("%.3f" % accuracy(y, run_cv(X,y,RF)))
print("K-nearest-neighbors:")
print("%.3f" % accuracy(y, run_cv(X,y,KNN)))

Logistic Regression:


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').